In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'

In [94]:
from IPython import get_ipython
get_ipython().run_line_magic('run', 'setup')
import gspread
import df2gspread as d2
import itertools

gc = gspread.service_account()
sh = gc.open_by_key('1OOXPXubXqnsOdNckUvVqt5M6b6q9oMwJMKhSbK4BQyk')
ws = sh.get_worksheet(0)

def publish(df):
  df = df.reset_index(level=0)
  sh.values_clear("'Regios'!A1:ZZ10000")
  sh.values_clear("'Regios'!A1:ZZ10000")
  ws.update([df.columns.values.tolist()] + df.values.tolist())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
@run('regios: download gemeenten en hun codes')
def cell():
  global gemeenten
  global bevolking
  gemeenten = pd.read_csv('gemeenten.csv')

  base = 'https://opendata.cbs.nl/ODataApi/OData/37230ned'

  # voor perioden pak de laatste
  periode = get_odata(base + '/Perioden').iloc[[-1]]['Key'].values[0]

  # startsWith would have been better to do in the filter but then the CBS "odata-ish" server responds with
  # "Object reference not set to an instance of an object."
  bevolking = get_odata(base + f"/TypedDataSet?$filter=(Perioden eq '{periode}')&$select=RegioS, BevolkingAanHetBeginVanDePeriode_1")
  # want de CBS odata API snap startsWith niet...
  bevolking = bevolking[bevolking.RegioS.str.startswith('GM')]
  # die _1 betekent waarschijnlijk dat het gedrag ooit gewijzigd is en er een nieuwe "versie" van die kolom is gepubliceerd
  bevolking.rename(columns={'RegioS': 'Code', 'BevolkingAanHetBeginVanDePeriode_1': 'BevolkingAanHetBeginVanDePeriode'}, inplace=True)
  bevolking.set_index('Code', inplace=True)
  
  gemeenten = gemeenten.merge(bevolking, how='left', left_on='Code', right_index=True)
  gemeenten.loc[gemeenten.Personen == 0, 'Personen'] = gemeenten.BevolkingAanHetBeginVanDePeriode
  gemeenten.set_index('Code', inplace=True)

  gemeenten = gemeenten[['Type', 'Landcode', 'GGD regio', 'Veiligheidsregio', 'Veiligheidsregio Code', 'Provincie', 'Landsdeel', 'Schoolregio', 'Personen', 'Opp land km2', 'Naam']]
  display(gemeenten)
  

regios: download gemeenten en hun codes
https://opendata.cbs.nl/ODataApi/OData/37230ned/Perioden
https://opendata.cbs.nl/ODataApi/OData/37230ned/TypedDataSet?$filter=(Perioden eq '2020MM11')&$select=RegioS, BevolkingAanHetBeginVanDePeriode_1


,Type,Landcode,GGD regio,Veiligheidsregio,Veiligheidsregio Code,Provincie,Landsdeel,Schoolregio,Personen,Opp land km2,Naam
Code,,,,,,,,,,,
GM0000,Gemeente,NL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,(niet gedefinieerd)
GM1680,Gemeente,NL,GGD Drenthe,Drenthe,VR03,Drenthe,Noord-Nederland,Noord,25366.0,276,Aa en Hunze
GM0358,Gemeente,NL,GGD Amsterdam,Amsterdam-Amstelland,VR13,Noord-Holland,West-Nederland,Noord,31957.0,20,Aalsmeer
GM0197,Gemeente,NL,GGD Noord- en Oost-Gelderland,Noord- en Oost-Gelderland,VR06,Gelderland,Oost-Nederland,Midden,27133.0,97,Aalten
GM0059,Gemeente,NL,GGD Fryslân,Fryslân,VR02,Friesland,Noord-Nederland,Noord,27881.0,102,Achtkarspelen
...,...,...,...,...,...,...,...,...,...,...,...
GM0879,Gemeente,NL,GGD West-Brabant,Midden- en West-Brabant,VR20,Noord-Brabant,Zuid-Nederland,Zuid,21995.0,121,Zundert
GM0301,Gemeente,NL,GGD Noord- en Oost-Gelderland,Noord- en Oost-Gelderland,VR06,Gelderland,Oost-Nederland,Midden,48105.0,41,Zutphen
GM1896,Gemeente,NL,GGD IJsselland,IJsselland,VR04,Overijssel,Oost-Nederland,Noord,22797.0,82,Zwartewaterland


In [79]:
@run('regios: RIVM cijfers ophalen')
def cell():
    datasets = [
        ('aantallen_gemeenten', 'COVID-19_aantallen_gemeente_per_dag', 0),
        ('ziekenhuisopnames', 'COVID-19_ziekenhuisopnames', 0),
        ('ziekenhuisopnames_gisteren', 'COVID-19_ziekenhuisopnames', 1),
    ]
    for df, dataset, day in datasets:
        globals()[df] = rivm_cijfers(dataset, day)
        # vervang lege gemeentecodes door de fallback 'GM0000'
        globals()[df]['Municipality_code'] = globals()[df]['Municipality_code'].fillna('GM0000')
        # knip de tijd van de datum af
        globals()[df].Date_of_report = globals()[df].Date_of_report.str.replace(' .*', '', regex=True)

regios: RIVM cijfers ophalen
downloads/COVID-19_aantallen_gemeente_per_dag-2021-01-14@14-15.csv exists
loading 0 downloads/COVID-19_aantallen_gemeente_per_dag-2021-01-14@14-15.csv
downloads/COVID-19_ziekenhuisopnames-2021-01-14@14-15.csv exists
loading 0 downloads/COVID-19_ziekenhuisopnames-2021-01-14@14-15.csv
loading 1 downloads/COVID-19_ziekenhuisopnames-2021-01-13@14-15.csv


In [99]:
@run('regios: absolute aantallen per gemeente')
def cell():
    def groepeer_op_gemeente(ag, columns):
        # simpele sum over groepering op gemeentecode, met rename
        df = ag.groupby(['Municipality_code'])[list(columns.keys())].sum()
        df.rename(columns=columns, inplace=True)
        return df

    positief_overleden = groepeer_op_gemeente(aantallen_gemeenten, {'Total_reported':'Positief getest', 'Deceased':'Overleden'})
    # beperk tot records op de datum van publicatie
    positief_overleden_toename = groepeer_op_gemeente(
        aantallen_gemeenten[aantallen_gemeenten.Date_of_report == aantallen_gemeenten.Date_of_publication],
        {'Total_reported':'Positief getest (toename)', 'Deceased':'Overleden (toename)'}
    )
    #print(positief_overleden.head())
    #print(positief_overleden_toename.head())
    
    admissions = groepeer_op_gemeente(ziekenhuisopnames, {'Hospital_admission':'Ziekenhuisopname'})
    admissions_gisteren = groepeer_op_gemeente(ziekenhuisopnames_gisteren, {'Hospital_admission':'Ziekenhuisopname_gisteren'})
    admissions_toename = admissions.merge(admissions_gisteren, how='left', on='Municipality_code')

    admissions_toename['Ziekenhuisopname (toename)'] = admissions_toename.Ziekenhuisopname - admissions_toename.Ziekenhuisopname_gisteren
    del admissions_toename['Ziekenhuisopname']
    del admissions_toename['Ziekenhuisopname_gisteren']

    # en plak het zwik aan elkaar
    global gemeenten
    new_columns = [df.columns.tolist() for df in [positief_overleden, positief_overleden_toename, admissions, admissions_toename]]
    new_columns = list(itertools.chain(*new_columns))
    gemeenten = (gemeenten[[col for col in gemeenten.columns if col not in new_columns and not 'per 100.000' in col and not '100k' in col]]
        .merge(positief_overleden, how='left', left_index=True, right_index=True)
        .merge(admissions, how='left', left_index=True, right_index=True)
        .merge(positief_overleden_toename, how='left', left_index=True, right_index=True)
        .merge(admissions_toename, how='left', left_index=True, right_index=True)
        .fillna(0)
    )

    # per 100k voor de absolute kolommen
    for df in [positief_overleden, admissions]:
        for col in df.columns:
            gemeenten[col + ' per 100.000'] = (gemeenten[col] * (100000 / gemeenten.Personen)).replace(np.inf, 0)
    
    days = (pd.to_datetime(aantallen_gemeenten['Date_of_publication']).max() - pd.to_datetime(aantallen_gemeenten['Date_of_publication']).min()) / np.timedelta64(1, 'D')
    gemeenten['Positief getest 1d/100k'] = gemeenten['Positief getest per 100.000'] / days
    display(gemeenten.head())
    publish(gemeenten.fillna(0))

regios: absolute aantallen per gemeente


,Type,Landcode,GGD regio,Veiligheidsregio,Veiligheidsregio Code,Provincie,Landsdeel,Schoolregio,Personen,Opp land km2,...,Positief getest,Overleden,Ziekenhuisopname,Positief getest (toename),Overleden (toename),Ziekenhuisopname (toename),Positief getest per 100.000,Overleden per 100.000,Ziekenhuisopname per 100.000,Positief getest 1d/100k
Code,,,,,,,,,,,,,,,,,,,,,
GM0000,Gemeente,NL,0,0,0,0,0,0,0.0,0,...,3740,32,1091,71,0,11,0.000000,0.000000,0.000000,0.000000
GM1680,Gemeente,NL,GGD Drenthe,Drenthe,VR03,Drenthe,Noord-Nederland,Noord,25366.0,276,...,711,8,9,13,0,0,2802.964598,31.538280,35.480565,8.704859
GM0358,Gemeente,NL,GGD Amsterdam,Amsterdam-Amstelland,VR13,Noord-Holland,West-Nederland,Noord,31957.0,20,...,1607,22,42,8,0,2,5028.632225,68.842507,131.426604,15.616870
GM0197,Gemeente,NL,GGD Noord- en Oost-Gelderland,Noord- en Oost-Gelderland,VR06,Gelderland,Oost-Nederland,Midden,27133.0,97,...,1200,17,41,11,0,1,4422.658755,62.654332,151.107507,13.734965
GM0059,Gemeente,NL,GGD Fryslân,Fryslân,VR02,Friesland,Noord-Nederland,Noord,27881.0,102,...,815,3,20,14,0,0,2923.137621,10.760016,71.733439,9.078067


In [89]:
r = ziekenhuisopnames[ziekenhuisopnames.Municipality_code == 'GM0599']['Hospital_admission'].sum()
ry = ziekenhuisopnames_gisteren[ziekenhuisopnames_gisteren.Municipality_code == 'GM0599']['Hospital_admission'].sum()
print(ry, r, r - ry)
gemeenten.dtypes

2312 2358 46


Type                             object
Landcode                         object
GGD regio                        object
Veiligheidsregio                 object
Veiligheidsregio Code            object
Provincie                        object
Landsdeel                        object
Schoolregio                      object
Personen                        float64
Opp land km2                      int64
Naam                             object
Positief getest                   int64
Overleden                         int64
Ziekenhuisopname                  int64
Positief getest (toename)         int64
Overleden (toename)               int64
Ziekenhuisopname (toename)        int64
Positief getest per 100.000     float64
Overleden per 100.000           float64
Ziekenhuisopname per 100.000    float64
Positief getest 1d/100k         float64
dtype: object